In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, f1_score, roc_auc_score, r2_score, mean_squared_error, mean_absolute_error
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
from nltk.metrics.distance import jaccard_distance
from nltk.tokenize import word_tokenize
import numpy as np

In [8]:
# Download NLTK resources
nltk.download('punkt') 
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
# Load spaCy's medium-sized English language model
nlp = spacy.load("en_core_web_md")

# Load the Universal Sentence Encoder model from TensorFlow Hub
# model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

In [10]:
# Calculate cosine similarity between student answer, examiner answer, and comprehension
def calculate_combined_similarity(student_answer, examiner_answer, comprehension, weights):
    # Check if any of the input text strings are empty
    if not student_answer or not examiner_answer or not comprehension:
        return 0.0  # Return zero similarity if any input text string is empty
    
    # Calculate similarity between student answer and examiner answer
    similarity_examiner = nlp(student_answer).similarity(nlp(examiner_answer))
    
    # Calculate similarity between student answer and comprehension
    similarity_comprehension = nlp(student_answer).similarity(nlp(comprehension))
    
    # Combine similarity scores using weights
    combined_similarity = (weights['examiner'] * similarity_examiner) + (weights['comprehension'] * similarity_comprehension)
    
    return combined_similarity

In [11]:
# # Function to calculate combined similarity using sentence embeddings
# def calculate_combined_similarity(student_answer, examiner_answer, comprehension, weights):
#     # Check if any of the input text strings are empty
#     if not student_answer or not examiner_answer or not comprehension:
#         return 0.0  # Return zero similarity if any input text string is empty
    
#     # Encode sentences to get their embeddings
#     student_embedding = model.encode(student_answer, convert_to_tensor=True)
#     examiner_embedding = model.encode(examiner_answer, convert_to_tensor=True)
#     comprehension_embedding = model.encode(comprehension, convert_to_tensor=True)
    
#     # Calculate cosine similarity between student answer and examiner answer
#     similarity_examiner = util.pytorch_cos_sim(student_embedding, examiner_embedding).item()
    
#     # Calculate cosine similarity between student answer and comprehension
#     similarity_comprehension = util.pytorch_cos_sim(student_embedding, comprehension_embedding).item()
    
#     # Combine similarity scores using weights
#     combined_similarity = (weights['examiner'] * similarity_examiner) + (weights['comprehension'] * similarity_comprehension)
    
#     return combined_similarity


In [12]:
# Preprocess text data
def preprocess_text(text):
    text = text.lower()
    text = ''.join([char for char in text if char.isalnum() or char.isspace()])
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)


In [13]:
# Load data
data = pd.read_csv('./Essay Grading Dataset JET.csv', encoding='Latin-1')
data.head


<bound method NDFrame.head of      Question_ID                                      Comprehension  \
0            1.0  In Natural Language Processing, Feature Extrac...   
1            1.0  In Natural Language Processing, Feature Extrac...   
2            1.0  So we know that machines can only understand n...   
3            1.0  So we know that machines can only understand n...   
4            1.0  If we ask any NLP practitioner or data scienti...   
..           ...                                                ...   
691          4.0  n\tSNOBOL ("StriNg Oriented and symBOlic Langu...   
692          4.0  n\tSNOBOL ("StriNg Oriented and symBOlic Langu...   
693          4.0  Portability: \nSo that the program can be move...   
694          4.0  Portability: \nSo that the program can be move...   
695          4.0  Portability: \nSo that the program can be move...   

                                              Question  \
0             What is Feature Extraction from the text   
1

In [14]:
# Preprocess data
data['Preprocessed_Examiner_Answer'] = data['Examiner_Answer'].apply(preprocess_text)

print(data['Preprocessed_Examiner_Answer'])


0      textual data data feed machine learning algori...
1      textual data data feed machine learning algori...
2      know machine understand number make machine ab...
3      know machine understand number make machine ab...
4      featue extraction difficult requires conversio...
                             ...                        
691    n snobol string oriented symbolic language ser...
692    n snobol string oriented symbolic language ser...
693    portability program moved new computer easily ...
694    portability program moved new computer easily ...
695    portability program moved new computer easily ...
Name: Preprocessed_Examiner_Answer, Length: 696, dtype: object


In [15]:
data['Preprocessed_Student_Answer'] = data['Student_Answer'].apply(preprocess_text)

print(data['Preprocessed_Student_Answer'])

0      process converting text data number called fea...
1                                        text processing
2      need feature extraction text convert text numb...
3                                           numeric form
4                                              take time
                             ...                        
691    snobol stand string oriented symbolic language...
692                                               snobol
693    portability program moved new computer easily ...
694    interoperability concern achieving functionali...
695                                          portability
Name: Preprocessed_Student_Answer, Length: 696, dtype: object


In [16]:
data['Preprocessed_Comprehension'] = data['Comprehension'].apply(preprocess_text)

print(data['Preprocessed_Comprehension'])

0      natural language processing feature extraction...
1      natural language processing feature extraction...
2      know machine understand number make machine ab...
3      know machine understand number make machine ab...
4      ask nlp practitioner data scientist answer yes...
                             ...                        
691    n snobol string oriented symbolic language ser...
692    n snobol string oriented symbolic language ser...
693    portability program moved new computer easily ...
694    portability program moved new computer easily ...
695    portability program moved new computer easily ...
Name: Preprocessed_Comprehension, Length: 696, dtype: object


In [35]:
# Specify weights for examiner answer and comprehension
weights = {'examiner': 0.9, 'comprehension': 0.1}

# Calculate semantic similarity
data['Semantic_Similarity'] = data.apply(lambda row: calculate_combined_similarity(row['Preprocessed_Student_Answer'], row['Preprocessed_Examiner_Answer'], row['Comprehension'], weights), axis=1)

# Display semantic similarity
print(data['Semantic_Similarity'])

C:\Users\USER\AppData\Local\Temp\ipykernel_9480\464760753.py:8: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_examiner = nlp(student_answer).similarity(nlp(examiner_answer))
C:\Users\USER\AppData\Local\Temp\ipykernel_9480\464760753.py:11: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_comprehension = nlp(student_answer).similarity(nlp(comprehension))


0      0.810328
1      0.781186
2      0.855375
3      0.691893
4      0.505492
         ...   
691    0.860276
692    0.000000
693    0.977304
694    0.966633
695    0.764786
Name: Semantic_Similarity, Length: 696, dtype: float64


In [36]:
# Define features and target
X = data[['Semantic_Similarity', 'Question_Score']] 
y = data['Student_Score']

print(data[['Semantic_Similarity', 'Question_Score', 'Student_Score']])
data[['Semantic_Similarity', 'Question_Score', 'Student_Score']].to_csv('semantic_similarity2.csv', index=False)



     Semantic_Similarity  Question_Score  Student_Score
0               0.810328              10              7
1               0.781186              10              0
2               0.855375              10              9
3               0.691893              10              0
4               0.505492              10              0
..                   ...             ...            ...
691             0.860276               4              2
692             0.000000               4              0
693             0.977304               3              2
694             0.966633               3              2
695             0.764786               3              0

[696 rows x 3 columns]


In [37]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Random Forest Regressor 

In [38]:
# Build and train the model
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [39]:
# Make predictions
y_pred_rf = rf_model.predict(X_test)
print(y_pred_rf)

[1.61       4.02       0.         2.         4.87       2.14
 3.92       8.49       5.91       1.12       0.88       1.
 4.55       3.03       3.78       4.06       4.         5.16
 0.         3.96       4.94       5.16       2.         4.82
 3.72       3.98       0.         2.89       4.69       3.92
 4.55       0.95       0.         3.72       0.33       6.01
 2.03       2.         3.22       0.         0.         3.77
 0.         2.96404762 1.22       0.02       0.02       0.
 3.72       2.         1.94       4.97       4.08       6.1
 0.99       6.         3.98       2.         6.         3.77
 3.98       0.88       4.7        0.02       3.65       2.
 4.9        2.09       2.         1.61       0.         8.49
 7.68       7.68       3.92       2.04       1.78430012 7.82
 2.         0.         0.         3.72       3.22       0.
 1.22       3.04       3.78       2.         4.87       4.08
 5.         6.86       1.24       4.92       0.         7.9
 1.         0.         4.97       

In [40]:
# Create a DataFrame to display actual and predicted values
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_rf})
print(results_df)

     Actual  Predicted
674       1       1.61
314       4       4.02
611       0       0.00
431       2       2.00
552       5       4.87
..      ...        ...
24        4       4.00
158       0       0.02
388       2       2.08
482       5       4.70
110       4       3.90

[140 rows x 2 columns]


In [41]:
# Evaluate the model
mse = mean_squared_error(y_test, y_pred_rf)
r2 = r2_score(y_test, y_pred_rf)
rmse = mean_squared_error(y_test, y_pred_rf, squared=False)
mae = mean_absolute_error(y_test, y_pred_rf)

print(f'Mean Squared Error: {mse}')
print(f'R-squared (R²): {r2}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'Mean Absolute Error (MAE): {mae}')

Mean Squared Error: 0.2737329962829779
R-squared (R²): 0.945552860013331
Root Mean Squared Error (RMSE): 0.5231949887785413
Mean Absolute Error (MAE): 0.19471608926787498


c:\Users\USER\Documents\WebProjects\EssayGradingBE\EssayGradingEnv\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


## Gradient Boosting Regressor

In [42]:
import xgboost as xgb

In [43]:
# Build and train the model
xgb_model = xgb.XGBRegressor(objective ='reg:squarederror', random_state=42)
xgb_model.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [44]:
# Make predictions
y_pred_xgb = xgb_model.predict(X_test)
print(y_pred_xgb)

[ 1.0237519e+00  4.0061002e+00 -1.4510278e-03  1.9996501e+00
  4.9943919e+00  2.0193257e+00  3.9747741e+00  8.9885015e+00
  5.9902873e+00  1.0239304e+00  4.5696747e-02  9.9976581e-01
  4.9796276e+00  3.0040393e+00  3.9889998e+00  4.0259790e+00
  3.9980440e+00  5.0026283e+00  6.5230671e-04  3.9934509e+00
  4.9724188e+00  5.0026283e+00  2.0005143e+00  4.9967089e+00
  3.9896164e+00  3.9812896e+00  6.3254690e-04  2.9859760e+00
  4.9907765e+00  3.9937496e+00  4.9796276e+00  9.9931407e-01
 -3.1344513e-03  3.9896164e+00  1.9949734e-02  5.9993958e+00
  2.0369029e+00  1.9979057e+00  3.0023031e+00  2.6092841e-05
  1.4413990e-03  3.9957314e+00 -2.3337700e-04  2.9997945e+00
  1.0301126e+00  8.2280906e-03  6.9004571e-04  6.0281004e-03
  4.0017209e+00  2.0002041e+00  1.9849889e+00  4.9993687e+00
  3.9991086e+00  5.9980311e+00  1.0004367e+00  5.9993958e+00
  3.9896164e+00  1.9946703e+00  5.9993958e+00  3.9957314e+00
  3.9953606e+00  4.5696747e-02  4.9796276e+00  8.2280906e-03
  3.0980332e+00  2.00559

In [45]:
# Create a DataFrame to display actual and predicted values
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_xgb})
print(results_df)

     Actual  Predicted
674       1   1.023752
314       4   4.006100
611       0  -0.001451
431       2   1.999650
552       5   4.994392
..      ...        ...
24        4   3.995361
158       0   0.000690
388       2   2.011442
482       5   4.979628
110       4   3.981339

[140 rows x 2 columns]


In [46]:
# Evaluate the model
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
r2 = r2_score(y_test, y_pred_xgb)
rmse = mean_squared_error(y_test, y_pred_xgb)
mae = mean_absolute_error(y_test, y_pred_xgb)
print(f'XGBoost Mean Squared Error: {mse_xgb}')
print(f'R-squared (R²): {r2}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'Mean Absolute Error (MAE): {mae}')


XGBoost Mean Squared Error: 0.021706448161472863
R-squared (R²): 0.9956824568550029
Root Mean Squared Error (RMSE): 0.021706448161472863
Mean Absolute Error (MAE): 0.024394906805199363


## Decision Tree Regressor

In [47]:
from sklearn.tree import DecisionTreeRegressor

In [48]:
# Build and train the Decision Tree Regressor model
dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(X_train, y_train)


DecisionTreeRegressor(random_state=42)

In [49]:
# Make predictions
y_pred_dt = dt_model.predict(X_test)
print(y_pred_dt)

[1.         4.         0.         2.         5.         2.
 4.         9.         6.         1.         0.         1.
 5.         3.         4.         4.         4.         5.
 0.         4.         5.         5.         2.         5.
 4.         4.         0.         3.         5.         4.
 5.         1.         0.         4.         0.         6.
 2.         2.         3.         0.         0.         4.
 0.         3.         1.         0.         0.         0.
 4.         2.         2.         5.         4.         6.
 1.         6.         4.         2.         6.         4.
 4.         0.         5.         0.         3.         2.
 5.         2.         2.         1.         0.         9.
 8.         8.         4.         2.         1.71428571 8.
 2.         0.         0.         4.         3.         0.
 1.         3.         4.         2.         5.         4.
 5.         7.         1.         5.         0.         8.
 1.         0.         5.         2.         3.         

In [50]:
# Create a DataFrame to display actual and predicted values
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_dt})
print(results_df)

     Actual  Predicted
674       1        1.0
314       4        4.0
611       0        0.0
431       2        2.0
552       5        5.0
..      ...        ...
24        4        4.0
158       0        0.0
388       2        2.0
482       5        5.0
110       4        4.0

[140 rows x 2 columns]


In [51]:
# Evaluate the model
mse = mean_squared_error(y_test, y_pred_dt)
rmse = mean_squared_error(y_test, y_pred_dt)
r2 = r2_score(y_test, y_pred_dt)
mae = mean_absolute_error(y_test, y_pred_dt)

print(f'Mean Squared Error: {mse}')
print(f'R-squared (R²): {r2}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'Mean Absolute Error (MAE): {mae}')

Mean Squared Error: 0.020991253644314867
R-squared (R²): 0.9958247133477245
Root Mean Squared Error (RMSE): 0.020991253644314867
Mean Absolute Error (MAE): 0.012244897959183673


### Dump Decision Tree Regressor

In [52]:
import joblib
joblib.dump(dt_model, 'dt_model_new.joblib')


['dt_model_new.joblib']